In [34]:
import datetime
HEADER="""INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
"""
ROW="( {user_id}, '{name}', '{birth_date}','{gender}', '{from_date}', '{to_date}', {location_id}, {segment_id} )"
RECORDS=1000
LOCATION_RANGE=(1,17) # 17 locations
SEGMENT_RANGE=(1,3) # 3 segments
FUTURE=datetime.date(2999,12,31)
GENDERS=["M","F","O"]
BIRTH_DATE_START=datetime.date(1950,1,1)
BIRTH_DATE_RANGE=365*55 # 55 years
SEGMENT_AGES_RANGES=[(0,25),(25,45),(45,100)] # 3 segments
INSERT_USERS_FILE="../15_insert_users.sql"

In [35]:
import faker
import random

faker_name = faker.Faker()
today = datetime.date.today()
insert_users = []
for i in range(1,RECORDS+1):
    name = faker_name.name()
    
    birth_date = BIRTH_DATE_START + datetime.timedelta(days=random.randint(0,BIRTH_DATE_RANGE))
    from_date = today - datetime.timedelta(days=random.randint(180,365*5))
    location_id = random.randint(*LOCATION_RANGE)
    gender = random.choice(GENDERS)
    age = (today - birth_date).days // 365
    age_segment = list(filter(lambda x : x[0] <= age < x[1], SEGMENT_AGES_RANGES))
    segment_id = random.randint(*SEGMENT_RANGE) + SEGMENT_AGES_RANGES.index(age_segment[0]) * 3
    insert_users.append(
        dict(
            user_id=i,
            name= name,
            birth_date= birth_date,
            from_date= from_date,
            to_date= FUTURE,
            gender=gender,
            location_id=location_id,
            segment_id=segment_id
        )
    )
    print(ROW.format(**insert_users[-1]))



( 1, 'Ryan Pearson', '1959-10-22','F', '2022-08-03', '2999-12-31', 15, 7 )
( 2, 'Robyn Daniels', '1974-10-30','F', '2022-02-21', '2999-12-31', 12, 8 )
( 3, 'Joseph Banks', '1962-09-14','M', '2021-04-07', '2999-12-31', 12, 9 )
( 4, 'Jasmine Park', '1955-11-02','M', '2022-11-30', '2999-12-31', 7, 9 )
( 5, 'Christopher Hall', '1953-02-02','F', '2023-06-18', '2999-12-31', 10, 7 )
( 6, 'Valerie Little', '1956-08-06','M', '2023-01-12', '2999-12-31', 11, 9 )
( 7, 'Jean Day', '1987-01-09','O', '2023-05-21', '2999-12-31', 1, 6 )
( 8, 'Jeanette Valencia', '1955-10-26','M', '2019-03-22', '2999-12-31', 12, 9 )
( 9, 'Felicia Mcmillan', '1951-11-09','F', '2020-04-18', '2999-12-31', 11, 7 )
( 10, 'Tammy Thomas', '1999-08-08','O', '2023-01-16', '2999-12-31', 11, 3 )
( 11, 'Megan Larson', '1972-07-19','F', '2021-03-16', '2999-12-31', 17, 7 )
( 12, 'Larry Nguyen', '1973-02-23','F', '2020-08-29', '2999-12-31', 15, 9 )
( 13, 'David Hernandez', '1970-11-14','F', '2019-09-25', '2999-12-31', 13, 8 )
( 14, 'J

In [36]:
## MAKE INSERT STATEMENT
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**user) for user in insert_users])
insert_sql += ';'
print(insert_sql)


INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
( 1, 'Ryan Pearson', '1959-10-22','F', '2022-08-03', '2999-12-31', 15, 7 ),
( 2, 'Robyn Daniels', '1974-10-30','F', '2022-02-21', '2999-12-31', 12, 8 ),
( 3, 'Joseph Banks', '1962-09-14','M', '2021-04-07', '2999-12-31', 12, 9 ),
( 4, 'Jasmine Park', '1955-11-02','M', '2022-11-30', '2999-12-31', 7, 9 ),
( 5, 'Christopher Hall', '1953-02-02','F', '2023-06-18', '2999-12-31', 10, 7 ),
( 6, 'Valerie Little', '1956-08-06','M', '2023-01-12', '2999-12-31', 11, 9 ),
( 7, 'Jean Day', '1987-01-09','O', '2023-05-21', '2999-12-31', 1, 6 ),
( 8, 'Jeanette Valencia', '1955-10-26','M', '2019-03-22', '2999-12-31', 12, 9 ),
( 9, 'Felicia Mcmillan', '1951-11-09','F', '2020-04-18', '2999-12-31', 11, 7 ),
( 10, 'Tammy Thomas', '1999-08-08','O', '2023-01-16', '2999-12-31', 11, 3 ),
( 11, 'Megan Larson', '1972-07-19','F', '2021-03-16', '2999-12-31', 17, 7 ),
( 12, 'Larry Nguyen', '1973-0

In [37]:
with open(INSERT_USERS_FILE, 'w') as f:
    f.write(insert_sql)

In [38]:
#### Set user subscriptions
HEADER="""INSERT INTO public."UserSubscription"(
    "UserId", "SubscriptionId", "ValidFrom", "ValidTo") VALUES
"""
ROW="( {user_id}, {subscription_id}, '{from_date}', '{to_date}' )"
SUBSCRIPTIONS_RANGE=(1,3) # 3 subscriptions
CHANGE_SUB_ONE_TIME=0.5
CHANGE_SUB_TWO_TIMES=0.3

In [39]:
# Set initial subs for every user, sharing same start date
insert_subscriptions = []
for user in insert_users:
    insert_subscriptions.append(
        dict(
            user_id=user['user_id'],
            subscription_id=random.randint(*SUBSCRIPTIONS_RANGE),
            from_date=user['from_date'],
            to_date=user['from_date'] + datetime.timedelta(days=round((datetime.date.today() - user['from_date']).days * random.random()))
        )
    )
    print(ROW.format(**insert_subscriptions[-1]))

( 1, 3, '2022-08-03', '2023-02-12' )
( 2, 1, '2022-02-21', '2022-12-19' )
( 3, 3, '2021-04-07', '2023-12-01' )
( 4, 3, '2022-11-30', '2023-04-20' )
( 5, 3, '2023-06-18', '2023-12-09' )
( 6, 2, '2023-01-12', '2023-07-23' )
( 7, 2, '2023-05-21', '2023-06-07' )
( 8, 1, '2019-03-22', '2023-09-30' )
( 9, 1, '2020-04-18', '2023-09-15' )
( 10, 3, '2023-01-16', '2023-02-24' )
( 11, 2, '2021-03-16', '2022-10-25' )
( 12, 1, '2020-08-29', '2020-11-17' )
( 13, 2, '2019-09-25', '2023-12-16' )
( 14, 1, '2020-10-26', '2021-02-17' )
( 15, 3, '2021-05-11', '2022-11-01' )
( 16, 1, '2021-10-01', '2022-04-14' )
( 17, 2, '2019-10-02', '2023-05-13' )
( 18, 1, '2020-05-21', '2021-09-09' )
( 19, 3, '2022-10-23', '2023-11-10' )
( 20, 1, '2020-04-08', '2023-06-15' )
( 21, 2, '2020-04-04', '2021-06-17' )
( 22, 2, '2021-08-26', '2021-12-30' )
( 23, 3, '2023-05-09', '2023-10-13' )
( 24, 1, '2022-05-12', '2022-11-25' )
( 25, 2, '2019-04-19', '2023-02-06' )
( 26, 1, '2023-01-14', '2023-03-09' )
( 27, 1, '2022-12-17'

In [40]:
lost_customers_index = random.sample(
    range(0,len(insert_subscriptions)), 
    int((CHANGE_SUB_ONE_TIME+CHANGE_SUB_TWO_TIMES)*len(insert_subscriptions))
)
len(lost_customers_index)


800

In [41]:
# Change subscription once
from collections import defaultdict


cut_index = int(CHANGE_SUB_ONE_TIME*len(insert_users))
for i in lost_customers_index:
    new_sub = insert_subscriptions[i]['subscription_id']
    while insert_subscriptions[i]['subscription_id'] == (new_sub := random.randint(*SUBSCRIPTIONS_RANGE)): ... 
    insert_subscriptions.append(
        dict(
            user_id=insert_subscriptions[i]['user_id'],
            subscription_id=new_sub ,
            from_date=insert_subscriptions[i]['to_date'] + datetime.timedelta(days=1),
            to_date=FUTURE
        )
    )
# Change subscription twice    
for i in lost_customers_index[cut_index:]:
    new_sub = insert_subscriptions[i]['subscription_id']
    while insert_subscriptions[i]['subscription_id'] == (new_sub := random.randint(*SUBSCRIPTIONS_RANGE)): ... 
    insert_subscriptions.append(
        dict(
            user_id=insert_subscriptions[i]['user_id'],
            subscription_id=new_sub ,
            from_date=insert_subscriptions[i]['to_date'] + datetime.timedelta(days=1),
            to_date=FUTURE
        )
    )

  

In [42]:
# Check users with miltiple rows
repeated_user_id_count = defaultdict(int)
for item in insert_subscriptions:
    print(repeated_user_id_count[item['user_id']])
    repeated_user_id_count[item['user_id']] += 1
count = [0,0,0]
for v in repeated_user_id_count.values():
    print(v)
    count[v-1] += 1
print(f'Users with 1 subscription: {count[0]}')
print(f'Users with 2 subscriptions: {count[1]}')
print(f'Users with 3 subscriptions: {count[2]}')  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [43]:
len(insert_subscriptions)

2100

In [44]:
## Create user subscriptions insert statement
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**sub) for sub in insert_subscriptions])
insert_sql += ';'   

print(insert_sql)

INSERT INTO public."UserSubscription"(
    "UserId", "SubscriptionId", "ValidFrom", "ValidTo") VALUES
( 1, 3, '2022-08-03', '2023-02-12' ),
( 2, 1, '2022-02-21', '2022-12-19' ),
( 3, 3, '2021-04-07', '2023-12-01' ),
( 4, 3, '2022-11-30', '2023-04-20' ),
( 5, 3, '2023-06-18', '2023-12-09' ),
( 6, 2, '2023-01-12', '2023-07-23' ),
( 7, 2, '2023-05-21', '2023-06-07' ),
( 8, 1, '2019-03-22', '2023-09-30' ),
( 9, 1, '2020-04-18', '2023-09-15' ),
( 10, 3, '2023-01-16', '2023-02-24' ),
( 11, 2, '2021-03-16', '2022-10-25' ),
( 12, 1, '2020-08-29', '2020-11-17' ),
( 13, 2, '2019-09-25', '2023-12-16' ),
( 14, 1, '2020-10-26', '2021-02-17' ),
( 15, 3, '2021-05-11', '2022-11-01' ),
( 16, 1, '2021-10-01', '2022-04-14' ),
( 17, 2, '2019-10-02', '2023-05-13' ),
( 18, 1, '2020-05-21', '2021-09-09' ),
( 19, 3, '2022-10-23', '2023-11-10' ),
( 20, 1, '2020-04-08', '2023-06-15' ),
( 21, 2, '2020-04-04', '2021-06-17' ),
( 22, 2, '2021-08-26', '2021-12-30' ),
( 23, 3, '2023-05-09', '2023-10-13' ),
( 24, 1, '

In [45]:
# Write to file
INSERT_SUBSCRIPTIONS_FILE="../16_insert_user_subscriptions.sql"
with open(INSERT_SUBSCRIPTIONS_FILE, 'w') as f:
    f.write(insert_sql)